In [1]:
import os
import sys

notebook_dir = os.path.abspath('')
sys.path.append(os.path.join(notebook_dir, '..'))

import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from keras_core import utils
from aepy.data.datasets import load_MNIST
from aepy.data.utils import evaluate, display_diff
from aepy.models.base.default_architectures import (VanillaEncoder,VanillaDecoder)
from aepy.models.cae.cae_model import CAE
from aepy.pipelines.training import TrainingPipeline

Using TensorFlow backend


2023-12-11 11:23:18.661194: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load MNIST dataset
x_train, y_train, x_test, y_test = load_MNIST(persistant=True)

#x_train = np.expand_dims(x_train, -1).astype("float32") / 255
#x_test = np.expand_dims(x_test, -1).astype("float32") / 255
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

# Obtaint number of clasess
n_classes = len(set(y_train))

# Convert labels to categorical
y_train = utils.to_categorical(y_train, n_classes)
y_test = utils.to_categorical(y_test, n_classes)

2023-12-11 11:23:21 [INFO]: train-images-idx3-ubyte.gz already exists.
2023-12-11 11:23:21 [INFO]: train-labels-idx1-ubyte.gz already exists.
2023-12-11 11:23:21 [INFO]: t10k-images-idx3-ubyte.gz already exists.
2023-12-11 11:23:21 [INFO]: t10k-labels-idx1-ubyte.gz already exists.


In [3]:
train_data = dict(data=x_train.astype(float), labels=y_train)
test_data = dict(data=x_test.astype(float), labels=y_test)

In [4]:
# Model creation
model = CAE(input_dim=x_train.shape[1], latent_dim=2,
            encoder=VanillaEncoder, decoder=VanillaDecoder, layers_conf=[32, 64])

In [5]:
pipe = TrainingPipeline(name='training_pipeline',
                        model=model, num_epochs=1)

trained_model = pipe(train_data=train_data)

2023-12-11 11:23:22 [INFO]: +++ training_pipeline +++
2023-12-11 11:23:22 [INFO]: Creating folder in ../output_dir/training_pipeline_2023-12-11_11-23-22


(128, 784)
(128, 784)
(128, 784)
(128, 784)


ValueError: in user code:

    File "/home/lucas/experiment/aepy/examples/../aepy/models/cae/cae_model.py", line 103, in train_step  *
        losses = self.compute_losses(x, outputs, labels_x)
    File "/home/lucas/experiment/aepy/examples/../aepy/models/cae/cae_model.py", line 74, in compute_losses  *
        contractive_loss = lambda_ *             tf.reduce_sum(tf.linalg.matmul(dh**2, tf.square(W)), axis=1)

    ValueError: Dimensions must be equal, but are 2 and 64 for '{{node MatMul}} = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false](pow, Square_1)' with input shapes: [128,2], [64,32].


In [ ]:
y_hat = trained_model.predict(test_data)

In [ ]:
print(x_test.shape)
print(y_hat['recon'].shape)
display_diff(x_test, y_hat['recon'])